In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
#import warnings
#warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
cat_df=pd.read_csv('categorical.csv')
cat_df.shape

(95412, 22)

In [3]:
num_df=pd.read_csv('numerical.csv')
num_df.shape

(95412, 315)

In [4]:
target_df=pd.read_csv('target.csv')
target_df.shape

(95412, 2)

In [8]:
data=pd.concat([cat_df,num_df,target_df],axis=1)
data

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B,TARGET_D
0,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,0,0.0
1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,0,0.0
2,NC,43,U,M,3,L,E,C,R,2,90,1,0,2,91,11,92,7,95,12,90,1,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10

In [9]:
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

# Apply the Random Forests algorithm but this time only by upscaling the data to deal with the imbalance.

In [10]:
X= data.drop(['TARGET_B'],axis=1)
y= data['TARGET_B']

In [11]:
num_X=X.select_dtypes(np.number)
cat_X=X.select_dtypes(np.object)


# One hot encoding for cat_x

In [12]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(cat_X)
encoded_categorical = encoder.transform(cat_X).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out()) # needed to avoid error
X = pd.concat([num_X, encoded_categorical], axis = 1)


In [13]:
# Note: we need to do train/test split before downsampling, and then only downsample the training set - Why?
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [14]:
from sklearn.utils import resample


In [15]:
# for upsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)
# quicker way to upsample category 1:

category_0 = trainset[trainset['TARGET_B'] == 0]
category_1 = trainset[trainset['TARGET_B'] == 1]
category_1 = resample(category_1,replace=True, n_samples = len(category_0))
print(category_0.shape)
print(category_1.shape)

trainset_new = pd.concat([category_0, category_1], axis = 0)
trainset_new = trainset_new.sample(frac =1) #randomize the rows
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']
# #data = data.reset_index(drop=True)
print(X_train.shape)

(72486, 356)
(72486, 356)
(144972, 355)


In [16]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

# Now we can remove the column target d from the set of features
X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=42)
clf.fit(X_train, y_train)
all_feat_train_score = clf.score(X_train, y_train)
print(all_feat_train_score)
all_feat_test_score = clf.score(X_test, y_test)
print(all_feat_test_score)

y_pred = clf.predict(X_test)
display(y_test.value_counts())
all_confusion_matrix = confusion_matrix(y_test, y_pred)
display(all_confusion_matrix)

0.6231134288000442
0.6125871194256668


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[11145,  6938],
       [  455,   545]], dtype=int64)

# Use Feature Selections that you have learned in class to decide if you want to use all of the features (Variance Threshold, RFE, PCA, etc.)


In [18]:
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object)

#ONE ENCODE CATEGORICALS
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out()) # needed to avoid error
X = pd.concat([numericalX, encoded_categorical], axis = 1)

#MIN-MAX NUMERICAL NORMALIZED WITH ENCODED_CATEGORICAL, NOW ARE DATAFRAME IT'S READY TO TAKE A RFE.
from sklearn.preprocessing import MinMaxScaler
transformer= MinMaxScaler().fit(numericalX)
numericalX = pd.DataFrame(transformer.transform(numericalX), columns=numericalX.columns)
X = pd.concat([numericalX, encoded_categorical], axis = 1)

#TO MAKE IT 100% RIGHT IT SHOULD HAVE BEEN SPLIT INTO TRAIN AND TEST, FIT THE MODEL WITH TR

## Variance Threshold

In [19]:
numericalX

,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_D
0,0.673077,1.0,0.333333,0.428571,0.0,0.381443,1.000000,0.772727,0.636364,0.863636,0.090909,0.0,1.000000,0.927083,0.909091,0.000000,0.608247,0.666667,1.000000,0.000000,0.000000,0.393939,0.343434,0.181818,0.101010,0.020202,0.011494,0.555556,0.010051,0.011108,0.009378,0.000000,0.353535,0.656566,0.474747,0.535354,0.929293,0.010101,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.113402,0.000000,0.000000,0.000000,0.464286,0.571429,0.607143,0.476190,0.595238,0.642857,0.333333,0.313131,0.424242,0.272727,0.111111,0.141414,0.181818,0.171717,0.131313,0.111111,0.151515,0.121212,0.111111,0.343434,0.252525,0.181818,0.262626,0.101010,0.232323,0.181818,0.333333,0.494949,0.282828,0.121212,0.040404,0.616162,0.070707,0.164384,0.191919,0.304615,0.394286,0.979798,0.959596,0.020202,0.020202,0.000000,0.000000,0.070707,0.070707,0.000000,0.079833,0.105833,0.230769,0.153846,0.868687,0.141414,0.969697,0.040404,0.070707,0.383838,0.808081,0.707071,0.323232,0.848485,0.161616,0.060606,0.04,0.050505,0.090909,0.151515,0.030303,0.226667,0.505051,0.252525,0.000000,0.000000,0.000000,0.020202,0.070707,0.131313,0.272727,0.474747,0.000000,0.010101,0.616162,0.682353,0.677778,0.245902,0.100,0.020202,0.000000,0.000000,0.141414,0.010101,0.000000,0.000000,0.020202,0.050505,0.171717,0.737374,0.000000,0.271889,0.774120,0.204667,0.212000,0.232667,0.252000,0.073818,0.131313,0.232323,0.232323,0.232323,0.151515,0.010101,0.00,0.000000,0.010101,0.040404,0.252525,0.242424,0.262626,0.171717,0.020202,0.00,0.000000,0.020202,0.282828,0.040404,0.515152,0.010101,0.464646,0.545455,0.030303,0.888889,0.080808,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.040404,0.010101,0.131313,0.155556,0.210526,0.020202,0.454545,0.565657,0.646465,0.505051,0.646465,0.444444,0.626263,0.535354,1.000000,0.000000,0.000000,0.090909,0.030303,0.080808,0.131313,0.090909,0.000000,0.054545,0.090909,0.030303,0.151515,0.191919,0.050505,0.040404,0.030303,0.000000,0.030303,0.414141,0.010101,0.000000,0.070707,0.131313,0.060606,0.050505,0.000000,0.059701,0.090909,0.040404,0.010101,0.030303,0.101010,0.020202,0.010101,0.070707,0.

In [21]:
from sklearn.feature_selection import VarianceThreshold 
var_threshold = 0.02
sel = VarianceThreshold(threshold=(var_threshold))

# 1) default is threshold = 0 :eliminate all coumns that are identical for all rows
# 2) in practice we would scale the columns first, and then apply threshold, or apply 
#    a different threshold for different columns

# This drops the columns that have a variance less than this threshold
sel = sel.fit(numericalX)
temp = sel.transform(numericalX)
temp = pd.DataFrame(temp)
print(numericalX.shape)
print(temp.shape)

(95412, 331)
(95412, 90)


In [22]:
pd.DataFrame(temp).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
0,0.673077,1.0,0.333333,0.428571,0.381443,1.000000,0.636364,0.090909,0.0,1.0,0.909091,0.608247,0.666667,1.000000,0.343434,0.181818,0.555556,0.0,0.353535,0.656566,0.929293,0.010101,0.494949,0.979798,0.959596,0.020202,0.000000,0.000000,0.079833,0.105833,0.230769,0.153846,0.868687,0.141414,0.070707,0.808081,0.707071,0.848485,0.505051,0.020202,0.070707,0.131313,0.272727,0.474747,0.000000,0.616162,0.000000,0.000000,0.000000,0.020202,0.050505,0.171717,0.737374,0.000000,0.271889,0.131313,0.515152,0.464646,0.545455,0.131313,0.454545,0.646465,0.646465,0.626263,0.535354,1.000000,0.0,0.343434,0.181818,0.747475,0.888889,0.777778,0.596154,0.141414,0.141414,0.313131,0.545455,0.464646,0.909091,0.101010,0.333333,0.656566,0.404040,0.285714,0.4,0.433333,0.498045,0.0,1.000000,0.622951
1,0.250000,1.0,0.000000,0.785714,0.536082,0.090909,0.818182,1.000000,0.0,1.0,0.818182,0.463918,0.833333,1.000000,0.555556,0.111111,1.000000,1.0,0.000000,0.000000,0.676768,0.000000,0.757576,1.000000,1.000000,0.000000,0.000000,0.000000,0.911333,0.869667,0.923077,0.769231,0.969697,0.040404,0.090909,0.949495,0.888889,0.959596,0.444444,0.979798,0.989899,0.989899,0.989899,1.000000,0.949495,0.838384,0.000000,0.000000,0.000000,0.919192,0.919192,0.919192,0.949495,0.478632,0.019969,0.020202,0.666667,0.565657,0.444444,0.020202,0.717172,0.838384,0.818182,0.646465,0.575758,1.000000,1.0,0.555556,0.111111,0.393939,0.505051,0.929293,0.192308,0.262626,0.565657,0.979798,1.000000,0.000000,0.969697,0.040404,1.000000,0.000000,1.000000,0.952381,0.8,0.183333,0.774510,0.0,0.333333,0.000000
2,0.807692,1.0,0.333333,0.500000,0.000000,0.090909,0.909091,0.545455,0.0,1.0,0.000000,0.624862,0.333333,0.111111,0.292929,0.333333,0.111111,0.0,0.020202,0.989899,0.969697,0.020202,0.444444,0.696970,0.696970,0.060606,0.050505,0.030303,0.082833,0.091000,0.153846,0.076923,0.787879,0.222222,0.181818,0.767677,0.656566,0.868687,0.606061,0.000000,0.010101,0.060606,0.181818,0.505051,0.000000,0.363636,0.020202,0.242424,0.030303,0.000000,0.020202,0.090909,0.444444,0.000000,0.431644,0.323232,0.313131,0.383838,0.626263,0.212121,0.616162,0.737374,0.707071,0.787879,0.626263,0.828283,1.0,0.282828,0.323232,0.848485,0.969697,0.656566,0.423077,0.121212,0.232323,0.505051,0.696970,0.313131,0.000000,0.353535,0.222222,0.777778,0.171717,0.428571,0.4,0.416667,0.078617,1.0,1.000000,0.967213
3,0.826923,1.0,0.333333,0.285714,0.288660,0.000000,0.909091,0.909091,0.0,1.0,0.090909,0.711340,0.000000,0.444444,0.141414,0.313131,0.000000,0.0,0.080808,0.929293,0.616162,0.000000,0.505051,0.858586,0.838384,0.040404,0.010101,0.000000,0.166667,0.210500,0.153846,0.076923,0.484848,0.525253,0.060606,0.737374,0.616162,0.848485,0.363636,0.101010,0.252525,0.505051,0.696970,0.929293,0.101010,0.424242,0.000000,0.090909,0.000000,0.010101,0.080808,0.171717,0.343434,0.997863,0.102919,0.272727,0.262626,0.464646,0.545455,0.171717,0.434343,0.818182,0.686869,0.333333,0.313131,0.000000,1.0,0.141414,0.313131,0.676768,0.565657,0.434343,0.730769,0.101010,0.191919,0.393939,0.454545,0.555556,0.454545,0.171717,0.232323,0.777778,0.222222,0.761905,0.4,0.433333,0.899764,1.0,1.000000,0.655738
4,0.288462,1.0,0.333333,0.214286,0.206186,0.000000,0.818182,0.000000,0.5,0.0,0.181818,0.793814,0.333333,0.222222,0.090909,0.535354,1.000000,1.0,0.000000,0.000000,0.020202,0.989899,0.575758,1.000000,0.989899,0.000000,0.000000,0.000000,0.096000,0.099000,0.307692,0.230769,0.909091,0.101010,0.000000,0.828283,0.494949,0.929293,0.010101,0.000000,0.010101,0.020202,0.161616,0.676768,0.000000,0.454545,0.000000,0.000000,0.000000,0.252525,0.585859,0.747475,0.838384,0.534188,0.195084,0.242424,0.111111,0.202020,0.808081,0.010101,0.767677,0.737374,0.656566,0.808081,0.313131,0.818182,1.0,0.090909,0.535354,0.656566,1.000000,0.454545,0.6538

In [23]:
# To check which columns were removed, you can manipulate the results
# from the code below. This gives the variance of each feature in order
# of appearance of the dataset.
sel.variances_


array([7.66186440e-02, 9.71103258e-02, 6.02931667e-02, 6.01836810e-02,
       3.29582408e-04, 5.18439428e-02, 7.57158344e-02, 1.48399740e-02,
       1.25407083e-01, 6.22566035e-03, 1.31956896e-01, 6.89087854e-02,
       1.48636119e-01, 1.11937388e-03, 1.30207180e-01, 1.75493674e-04,
       2.21932083e-02, 8.02275908e-02, 9.03764075e-02, 1.49132173e-03,
       2.61802339e-03, 1.34243682e-02, 2.32567811e-02, 3.19975201e-02,
       1.96784193e-03, 2.68155697e-03, 2.28104662e-03, 1.02613600e-01,
       3.38581196e-03, 3.76408939e-03, 3.59628980e-03, 2.29533182e-01,
       9.94600474e-02, 1.63556065e-01, 3.16042737e-03, 3.39492297e-03,
       4.50544144e-02, 2.84235197e-02, 1.22804261e-03, 5.09957894e-03,
       1.93912343e-02, 9.53325826e-04, 9.61133151e-04, 6.55013636e-04,
       1.18787514e-03, 4.75926813e-04, 5.03014665e-04, 2.63911441e-04,
       1.36515733e-02, 5.14425138e-04, 1.56917030e-03, 1.53123652e-03,
       9.84719265e-03, 9.64517465e-03, 9.31928327e-03, 7.47231939e-03,
      

In [24]:
#This gives you the result as True and False for the columns that we
# selected and those which were removed, respectively.
sel.variances_ > var_threshold
sel.get_support()
var_list = list(sel.get_support())
var_list


[True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 Tru

In [27]:
removed_columns = pd.DataFrame(data=(numericalX.columns,sel.variances_,sel.get_support()), index=('column_name','variance','statement')).T
removed_columns.loc[(removed_columns['statement'] == False),:]

,column_name,variance,statement
4,ODATEW_MM,0.000329582,False
7,MINRDATE_YR,0.01484,False
9,MAXRDATE_YR,0.00622566,False
13,FIRSTDATE_YR,0.00111937,False
15,TCODE,0.000175494,False
...,...,...,...
322,MAXRAMNT,2.51858e-05,False
323,LASTGIFT,0.000194785,False
324,TIMELAG,5.10942e-05,False
325,AVGGIFT,0.00011629,False


In [ ]:
# So these are the 241 columns we can drop
# if we also want to see the actual variances we can use this:

# RFE - Recursive feature elimination

In [31]:
X = numericalX
y = target_df['TARGET_B']

from sklearn.feature_selection import RFE # RFE- Recursive Feature Elimination
from sklearn import linear_model
lm = linear_model.LinearRegression()
rfe = RFE(lm, n_features_to_select=20, verbose=False)
rfe.fit(X, y)

RFE(estimator=LinearRegression(), n_features_to_select=20, verbose=False)

In [32]:
rfe.ranking_

array([306, 301, 207, 305,   1, 237, 266,  34, 295, 143, 285, 184, 205,
        33, 241,  24, 201, 243, 264, 302, 169, 280, 279, 218, 291, 232,
       168, 265, 284,  25,  26, 111, 112, 113,  44,  45,   1,  31, 263,
       136,  11, 202, 137, 251, 135, 277, 204, 140,  12,  14,  13,  10,
        53,  57, 267, 103,  42, 216,  58,  23,  21,  22,  46,  43,  36,
        47,  35,  48, 102,  18,  16,  17,  20,  19, 214, 286,  80,   2,
        40,  41,  84,  60,  79,  56,  52, 151,  51, 258,  59,  72, 292,
         1,   1,   1, 145,  49,  50,  81, 287, 224, 220, 221,  88,  87,
        27,  28, 240,  63,   4,   5,  62,  95,  85,   6,  15,  75,   1,
         3,   7,   1,   1,   1,  32,  29,  30, 312, 195, 252, 261, 254,
       253, 105, 101, 219, 100,  39,  99,  55,  54,  71, 181, 187, 146,
        64, 308, 299, 296, 309, 300, 215, 141, 114, 107, 115,  93,  37,
       171, 117, 304, 185, 170,  89,  90,   9,  38, 153, 154, 155, 116,
        94, 206,  91,   8,  92, 191, 274, 190, 255, 230, 262, 19

In [37]:

feature_ranking = rfe.support_
feature_ranking 

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,  True,  True,  True, False, False, False, False,
       False, False,

In [38]:
selected_columns = X.columns[feature_ranking]
selected_features_X = X[selected_columns]
selected_features_X

,ODATEW_MM,ETH1,DW3,DW4,DW5,HHD10,ETHC1,ETHC2,ETHC3,TPE3,TPE4,TPE5,TPE6,RAMNTALL,NGIFTALL,MINRAMNT,LASTGIFT,TIMELAG,AVGGIFT,TARGET_D
0,0.0,0.929293,0.020202,0.020202,0.000000,0.090909,0.226667,0.505051,0.252525,0.000000,0.000000,0.0,0.0,0.023965,0.127119,0.005,0.010,0.003676,0.006465,0.00
1,0.0,0.676768,0.000000,0.000000,0.000000,0.050505,0.240000,0.444444,0.050505,0.000000,0.000000,0.0,0.0,0.003590,0.008475,0.010,0.025,0.016544,0.014399,0.00
2,0.0,0.969697,0.010101,0.060606,0.050505,0.111111,0.226667,0.606061,0.181818,0.000000,0.000000,0.0,0.0,0.019954,0.110169,0.002,0.005,0.011029,0.006204,0.00
3,0.0,0.616162,0.030303,0.040404,0.010101,0.212121,0.173333,0.363636,0.131313,0.000000,0.000000,0.0,0.0,0.010135,0.063559,0.002,0.010,0.008272,0.005534,0.00
4,0.0,0.020202,0.000000,0.000000,0.000000,0.090909,0.013333,0.010101,0.000000,0.010101,0.010101,0.0,0.0,0.025443,0.152542,0.003,0.015,0.012868,0.005586,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,0.0,0.787879,0.080808,0.323232,0.242424,0.161616,0.253333,0.555556,0.050505,0.020202,0.020202,0.0,0.0,0.001267,0.000000,0.025,0.025,0.008272,0.023745,0.00
95408,0.0,0.919192,0.010101,0.010101,0.000000,0.070707,0.373333,0.585859,0.050505,0.040404,0.040404,0.0,0.0,0.000739,0.000000,0.020,0.020,0.008272,0.018738,0.00
95409,0.0,0.828283,0.010101,0.020202,0.010101,0.212121,0.266667,0.494949,0.121212,0.000000,0.000000,0.0,0.0,0.004751,0.025424,0.003,0.010,0.002757,0.007009,0.00
95410,0.0,0.929293,0.000000,0.000000,0.000000,0.090909,0.253333,0.565657,0.161616,0.000000,0.000000,0.0,0.0,0.051204,0.169492,0.005,0.018,0.003676,0.010875,0.09


# Re-run the Random Forest algorithm to determine if the Feature Selection has improved the results.

In [39]:
y1 = data['TARGET_B']
X1 = selected_features_X

In [40]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

In [41]:
# for upsampling we need to temporarily concat X_train and y_train
trainset1 = pd.concat([X1_train, y1_train], axis=1)
# quicker way to upsample category 1:

category_0_1 = trainset1[trainset1['TARGET_B'] == 0]
category_1_1 = trainset1[trainset1['TARGET_B'] == 1]
category_1_1 = resample(category_1_1,replace=True, n_samples = len(category_0_1))
print(category_0_1.shape)
print(category_1_1.shape)

trainset_new_1 = pd.concat([category_0_1, category_1_1], axis = 0)
trainset_new_1 = trainset_new_1.sample(frac =1) #randomize the rows
X1_train = trainset_new_1.drop(['TARGET_B'], axis=1)
y1_train = trainset_new_1['TARGET_B']
# #data = data.reset_index(drop=True)
print(X1_train.shape)

(72486, 21)
(72486, 21)
(144972, 20)


In [42]:
X1_train = pd.DataFrame(X1_train)
X1_test = pd.DataFrame(X1_test)

y1_train_regression = X1_train['TARGET_D']
y1_test_regression = X1_test['TARGET_D']

# Now we can remove the column target d from the set of features
X1_train = X1_train.drop(['TARGET_D'], axis = 1)
X1_test = X1_test.drop(['TARGET_D'], axis = 1)

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf1 = RandomForestClassifier(max_depth=5, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=42)
clf1.fit(X1_train, y1_train)
selected_train_score = clf1.score(X1_train, y1_train)
print(selected_train_score)
selected_test_score = clf1.score(X1_test, y1_test)
print(selected_test_score)

y1_pred = clf1.predict(X1_test)
selected_confusion_matrix = confusion_matrix(y1_test, y1_pred)
display(y1_test.value_counts())
display(selected_confusion_matrix)

0.5928317192285407
0.6117486768327831


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[11167,  6916],
       [  493,   507]], dtype=int64)

# Discuss the output and its impact in the business scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the business?

In [44]:
all_feat_train_score = clf.score(X_train, y_train)
print(all_feat_train_score)
all_feat_test_score = clf.score(X_test, y_test)
print(all_feat_test_score)

display(y_test.value_counts())
all_confusion_matrix = confusion_matrix(y_test, y_pred)
display(all_confusion_matrix)

0.6231134288000442
0.6125871194256668


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[11145,  6938],
       [  455,   545]], dtype=int64)

In [45]:
selected_train_score = clf1.score(X1_train, y1_train)
print(selected_train_score)
selected_test_score = clf1.score(X1_test, y1_test)
print(selected_test_score)

selected_confusion_matrix = confusion_matrix(y1_test, y1_pred)
display(y1_test.value_counts())
display(selected_confusion_matrix)

0.5928317192285407
0.6117486768327831


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[11167,  6916],
       [  493,   507]], dtype=int64)